### **Install and Imports**

In [ ]:
# !pip install numpy emoji langdetect contractions beautifulsoup4


In [1]:
#Import pandas library for data processing data file
import pandas as pd
from bs4 import BeautifulSoup
import csv
import os
import numpy as np
import re
import emoji
import textwrap # textwrap library for formatting text
from langdetect import detect, DetectorFactory
from collections import Counter
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import contractions 
from langdetect import detect, detect_langs, DetectorFactory
import re
import pandas as pd
from collections import Counter
from emoji import EMOJI_DATA
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
import nltk

In [2]:
pd.set_option("display.max_colwidth", None)  # show full column content

### **Read from CSV file**


In [3]:
df = pd.read_csv('2024_Sample_Complete_Clean.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481427 entries, 0 to 481426
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tweet   481427 non-null  object
dtypes: object(1)
memory usage: 3.7+ MB


In [4]:
#rename cloumn text to tweet 
df = df.rename(columns={'text': 'tweet'})

**Extract and remove hashtags**

In [5]:
# Regex patterns
urlPattern = re.compile(r'https?://\S+')
mentionPattern = re.compile(r'@\w+')
# save hashtage for each tweet
df['hashtags'] = df['tweet'].apply(lambda x: re.findall(r"#\w+", x))
# just drop the # symbol but keep the text
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r"#(\w+)", r"\1", str(x)).strip())


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481427 entries, 0 to 481426
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   tweet     481427 non-null  object
 1   hashtags  481427 non-null  object
dtypes: object(2)
memory usage: 7.3+ MB


**Remove URL and USER Names**

In [7]:
# function to remove @usernames with @USER and URLs with whitespace
def removeCompleteUsernamesURL(text: str) -> str:
    text = mentionPattern.sub("", str(text))
    text = urlPattern.sub("", text)
    return text

In [8]:
# function to replaces @usernames with @USER and URLs with @HTTPURL.
def removeUsernamesURL(text: str) -> str:
    text = mentionPattern.sub("@USER", str(text))
    text = urlPattern.sub("@HTTPURL", text)
    return text


In [9]:
df["cleanMaskedU_tweet"] = df["tweet"].apply(removeUsernamesURL)
df["cleanMaskedU_tweet"].head(5)
df["cleanNoU_tweet"] = df["tweet"].apply(removeCompleteUsernamesURL)
df["cleanNoU_tweet"].head(5)

0                                                                                                                                                          What if it was actually biden biting the secret service agents and they just blamed the dogs instead?
1    Welcome to Kamala Harris' America. Where Islamic Terrorists are allowed to cross the border, buy guns, them murder innocent people while shouting "allahu akbar". KamalaDumpsterFire Kamala Surge DemocratsHateAmerica VoteTrumpToSaveAmerica DeportThemAll
2                   I was just talking to my wife about that, when Kamala Harris said she wouldn't change anything in the past 3 1/2 years versus Trump telling all about the mistakes made his 1st term and won't make them again, that's apparent, thank God 🙏
3                                                                                                                                                                          I would take Donald Trump's family values and the way he g

In [10]:
#function to join lemmatized tokens back into a processed string # (Deepak, 2017; GeeksforGeeks, 2024b)
def joinTokens(df, lem_col="cleanMaskedU_tweet"):
    df["tweets_processed"] = df[lem_col].apply(lambda x: " ".join(x))
    return df

In [11]:
#  tokenize tweets and store results in a new column 'tokens'.
def tokenizeTweets(df, text_col="cleanMaskedU_tweet"): 
    df["tokens"] = df[text_col].apply(lambda x: word_tokenize(str(x))) # (Lane, 2021)
    return df

In [12]:
# POS helper — maps POS tags to WordNet categories
def get_pos(tag): # (Deepak, 2017; GeeksforGeeks, 2024b)
    if tag.startswith("V"):
        return wn.VERB
    elif tag.startswith("R"):
        return wn.ADV
    elif tag.startswith("J"):
        return wn.ADJ
    else:
        return wn.NOUN

# Function to clean, lemmatize, and rejoin one text entry
def processTweets(text):
    # 1. Tokenize
    tokens = word_tokenize(str(text))

    # # 2. Remove stopwords but keep ideological terms
    # stopwords_list = set(stopwords.words("english"))
    # ideological_terms = {
    #     "left","right","liberal","liberals","conservative","conservatives",
    #     "moderate","centrist","progressive","progressives","populist","populists",
    #     "socialist","socialists","capitalist","capitalists","marxist","communist",
    #     "democrat","democrats","republican","republicans","libertarian","authoritarian",
    #     "patriot","patriotic","nationalist","nationalists","globalist","globalists",
    #     "feminist","feminists","woke","antiwoke","antifa","alt","rightwing","leftwing",
    #     "religious","christian","muslim","jewish","secular","faith","family","patriotism",
    #     "abortion","gun","guns","climate","climatechange","environment","immigration",
    #     "border","security","tax","taxes","inflation","economy","healthcare","covid",
    #     "mask","vaccine","freedom","rights","justice","inequality","race","racism",
    #     "lgbt","gender","transgender","education","crime","police","defund","war",
    #     "ukraine","israel","palestine","terrorism","peace","prolife","choice",
    #     "studentdebt","minimumwage","unions","labor","wealth","elites","media"
    # }
    # stopwords_list -= ideological_terms
    # tokens = [t for t in tokens if t.lower() not in stopwords_list]

    # 3. Lemmatize (POS-aware) # (Deepak, 2017; GeeksforGeeks, 2024b)
    wordLem = WordNetLemmatizer()
    tagged = nltk.pos_tag(tokens)
    lemmas = [wordLem.lemmatize(w, get_pos(p)) for w, p in tagged]

    # 4. Join back into a cleaned string
    return " ".join(lemmas)


In [13]:
df = tokenizeTweets(df)
df["cleanMaskedU_tweet"] = df["cleanMaskedU_tweet"].apply(processTweets)
df = joinTokens(df)


**Emoticons**

In [21]:
for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(str).apply(emoji.demojize) # (carpedm20, 2025)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481427 entries, 0 to 481426
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   tweet             481427 non-null  object
 1   hashtags          481427 non-null  object
 2   text              481427 non-null  object
 3   cleanNoU_tweet    481427 non-null  object
 4   tokens            481427 non-null  object
 5   tweets_processed  481427 non-null  object
dtypes: object(6)
memory usage: 22.0+ MB


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481427 entries, 0 to 481426
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   tweet             481427 non-null  object
 1   hashtags          481427 non-null  object
 2   text              481427 non-null  object
 3   cleanNoU_tweet    481427 non-null  object
 4   tokens            481427 non-null  object
 5   tweets_processed  481427 non-null  object
dtypes: object(6)
memory usage: 22.0+ MB


#### 2020 SAVE 

In [15]:
# ##SAVE TO CSV
# df.to_csv(
#     "For Analysis/2020_Sample_Ana_FF_Clean.csv",
#     index=False,
#     encoding="utf-8"
# )

# df = df.rename(columns={"cleanMaskedU_tweet": "text"})
# df['text'].to_csv(
#     "For Analysis/2020_Sample_Ana_Text_Clean.csv",
#     index=False
# )

In [16]:
# see = pd.read_csv("For Analysis/2020_Sample_Ana_Text_Clean.csv")
# see.head(20)    

#### 2024 SAVE 

In [24]:
##SAVE TO CSV
df.to_csv(
    "For Analysis/2024_Sample_Ana_FF_Clean.csv",
    index=False,
    encoding="utf-8"
)

df = df.rename(columns={"cleanMaskedU_tweet": "text"})
df['text'].to_csv(
    "For Analysis/2024_Sample_Ana_Text_Clean.csv",
    index=False
)

In [25]:
see = pd.read_csv("For Analysis/2024_Sample_Ana_Text_Clean.csv")
see.head(20)    

,text
0,@ USER What if it be actually biden bite the secret service agent and they just blame the dog instead ?
1,"Welcome to Kamala Harris ' America . Where Islamic Terrorists be allow to cross the border , buy gun , them murder innocent people while shout `` allahu akbar '' . KamalaDumpsterFire Kamala Surge DemocratsHateAmerica VoteTrumpToSaveAmerica DeportThemAll"
2,"@ USER @ USER I be just talk to my wife about that , when Kamala Harris say she would n't change anything in the past 3 1/2 year versus Trump tell all about the mistake make his 1st term and wo n't make them again , that 's apparent , thank God :folded_hands:"
3,@ USER @ USER I would take Donald Trump 's family value and the way he govern over Kamala any day .
4,NFL legend give angry response to claim he endorse Kamala Harris:clapping_hands::clapping_hands: @ HTTPURL
5,"Give me a fourth Obama term than a second Trump term any day . By the way , Kamala Harris will do just fine ."
6,@ USER Donald Trump in his natural state the ... trash :wastebasket: @ HTTPURL
7,@ USER Because he 's support Donald Trump in thus election ?
8,"As we hand out candy on Halloween , Trump make 3 campaign stop TODAY ! The man be relentlessly work hard for our vote . Que Mala see the Obamas & amp ; Biden get rich & amp ; want the same without the work just like her predecessor . Current rally , another later . @ HTTPURL"
9,My name be Nick and I fully state Donald Trump be a terrible person and should never be in the White House ever again . AmericaNeedsTrump NOT


In [26]:
data = pd.read_csv("For Analysis/2024_Sample_Ana_Text_Clean.csv")
# randomly sample 270000 rows
data_sampled = data.sample(n=270000, random_state=42)
data_sampled.to_csv(
    "For Analysis/2024_Sample_Ana_Text_Clean_Sampled.csv",
    index=False
)